In [ ]:
dbutils.widgets.text("source", "", "Source from which to copy file from")
dbutils.widgets.text("dest", "", "Destination where to copy file to")
dbutils.widgets.text("py-files", "", "Additional comma separated list of files normally submitted via --py-files")

In [ ]:
import os
import uuid
import shutil

In [ ]:
class OperationFailed(RuntimeError):
  pass

In [ ]:
source = dbutils.widgets.get("source")
dest = dbutils.widgets.get("dest")
prefixed_dest = dest
py_files = dbutils.widgets.get("py-files").split(",")
if prefixed_dest.startswith("/") or prefixed_dest.startswith(".") or prefixed_dest.startswith("~"):
  prefixed_dest = f"file://{prefixed_dest}"
py_files = [f"{'file://' if f.startswith('/') or f.startswith('.') or f.startswith('~') else ''}{f}" for f in py_files]

In [ ]:
home_dir = os.path.expanduser('~')
print(f"Home directory is {home_dir}")
dest = dest.replace("~", home_dir)
prefixed_dest = prefixed_dest.replace("~", home_dir)
py_files = [f.replace("~", home_dir) for f in py_files]

In [ ]:
path_splits = os.path.split(source)
filename = path_splits[1]

In [ ]:
os.makedirs(dest, 0o770, True)

In [ ]:
dbutils.fs.cp(source, f"{prefixed_dest}/{filename}", True)

In [ ]:
os.chmod(f'{dest}', 0o770)

In [ ]:
import zipfile

with zipfile.ZipFile(f'{dest}/{filename}', 'r') as zip:
  zip.extractall(f'{dest}')

In [ ]:
for f in py_files:
  dbutils.fs.ls(f) # check if file exists because addPyFile() does nothing if it doesn't or if it can't be accessed
  spark.sparkContext.addPyFile(f)

In [ ]:
display(dbutils.fs.ls(f'{dest}'))